<a href="https://colab.research.google.com/github/prof-rossetti/intro-to-python/blob/main/notes/apis/Spotify_API_Demo_(2024).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# References



## The Spotify API

  + https://developer.spotify.com/documentation/web-api/
  + https://developer.spotify.com/documentation/general/guides/authorization-guide
  + https://developer.spotify.com/documentation/general/guides/scopes/

## The `spotipy` Package

The `spotipy` package provides an interface into the Spotify API.

  + https://github.com/plamere/spotipy
  + https://github.com/plamere/spotipy#quick-start
  + https://spotipy.readthedocs.io/en/latest/
  + https://spotipy.readthedocs.io/en/latest/#client-credentials-flow
  + https://spotipy.readthedocs.io/en/latest/#authorization-code-flow




# Setup



Installing the `spotipy` package into the notebook environment:

In [30]:
%%capture
!pip install spotipy

In [31]:
!pip list | grep spotipy

spotipy                          2.23.0


Create a [Spotify API Client application](https://developer.spotify.com/dashboard/applications/), note its credentials, then set them as notebook secrets called `SPOTIPY_CLIENT_ID` and `SPOTIPY_CLIENT_SECRET` respectively.

In [ ]:
from google.colab import userdata

SPOTIPY_CLIENT_ID = userdata.get("SPOTIPY_CLIENT_ID")
SPOTIPY_CLIENT_SECRET = userdata.get("SPOTIPY_CLIENT_SECRET")

# Usage

## Info Inputs

In [ ]:
search_term = "Dua Lipa"

### Info Processing

In [ ]:

from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials

creds = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
client = Spotify(client_credentials_manager=creds)
print("CLIENT:", type(client))

CLIENT: <class 'spotipy.client.Spotify'>


In [ ]:
# api request
results = client.search(q=search_term, limit=20)
print(results.keys())

dict_keys(['tracks'])


In [ ]:
print(results["tracks"].keys())

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])


In [ ]:
tracks = results["tracks"]["items"]
print(tracks[0].keys())

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])


In [ ]:
print(tracks[0].keys())

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])


In [ ]:
try:
    del tracks[0]["album"]["available_markets"]
except:
    pass

In [ ]:
# convert image URL to html
# credit to: https://towardsdatascience.com/rendering-images-inside-a-pandas-dataframe-3631a4883f60

def img_html(url):
    return '<img src="'+ url + '" width="50" >'

def preview_html(url):
    if url:
        return '<a href="'+ url + '" >Listen on Spotify</a>'
    else:
        return None

In [ ]:
records = []
# parsing the response
for index, track in enumerate(tracks):
    #print(' ', index, "|", track['name'], "|", track["artists"][0]["name"])
    record = {
        "index": index,
        "name": track['name'],
        "artist": track["artists"][0]["name"],
        #"duration_ms": track['duration_ms'],
        #"explicit":  track['explicit'],
        "popularity": track["popularity"],
        "preview_url": preview_html(track["preview_url"]),
        "album_art": img_html(track["album"]["images"][0]["url"])
    }
    records.append(record)

In [ ]:
from pandas import DataFrame

tracks_df = DataFrame(records)
tracks_df.head()

,index,name,artist,popularity,preview_url,album_art
0,0,Levitating,Dua Lipa,79,"<a href=""https://p.scdn.co/mp3-preview/ac28d1b...","<img src=""https://i.scdn.co/image/ab67616d0000..."
1,1,Dua Lipa,Jack Harlow,69,"<a href=""https://p.scdn.co/mp3-preview/a67e101...","<img src=""https://i.scdn.co/image/ab67616d0000..."
2,2,Illusion,Dua Lipa,87,"<a href=""https://p.scdn.co/mp3-preview/6b05342...","<img src=""https://i.scdn.co/image/ab67616d0000..."
3,3,Houdini,Dua Lipa,92,"<a href=""https://p.scdn.co/mp3-preview/df4af86...","<img src=""https://i.scdn.co/image/ab67616d0000..."
4,4,Training Season,Dua Lipa,92,"<a href=""https://p.scdn.co/mp3-preview/5608f57...","<img src=""https://i.scdn.co/image/ab67616d0000..."


In [ ]:
from IPython.core.display import HTML

# displaying the dataframe as HTML, with HTML links and images:
tracks_table = HTML(tracks_df.to_html(escape=False, index=False, formatters=dict(Icon=img_html)))

### Info Outputs

In [ ]:
display(tracks_table)

index,name,artist,popularity,preview_url,album_art
0,Levitating,Dua Lipa,79,Listen on Spotify,
1,Dua Lipa,Jack Harlow,69,Listen on Spotify,
2,Illusion,Dua Lipa,87,Listen on Spotify,
3,Houdini,Dua Lipa,92,Listen on Spotify,
4,Training Season,Dua Lipa,92,Listen on Spotify,
5,New Rules,Dua Lipa,81,Listen on Spotify,
6,IDGAF,Dua Lipa,79,Listen on Spotify,
7,Don't Start Now,Dua Lipa,81,Listen on Spotify,
8,Dance The Night - From Barbie The Album,Dua Lipa,88,Listen on Spotify,
9,Illusion - Extended,Dua Lipa,69,Listen on Spotify,
